## The Saltenis estimator

In [1]:
%matplotlib inline
from Functions import A1, A2, B1, B2, B3, C1, C2
import numpy as np
import pandas as pd
import sobol_seq
import matplotlib.pyplot as plt
from pandas import ExcelWriter
plt.style.use('ggplot')

### Defining the test functions

In [2]:
k = 6

a2 = np.array([0,0.5,3,9,99,99])
b3 = np.array([6.42,6.42,6.42,6.42,6.42,6.42])

functions = [A1, A2, B1, B2, B3, C1, C2]

AE_df = pd.read_excel('AE_df.xlsx',index_col=0) #the analytical values

def scrambler(q,s):
    return q+s

### And the analytical values

In [3]:
p = 14
run = 50

n = [2,3,4,6]

df_r = pd.DataFrame(sobol_seq.i4_sobol_generate(6*k,-1+2**p))

df_index = pd.read_csv('2019.10.08_index.csv', index_col=0)

df = pd.concat([df_r.T.reindex(df_index.iloc[r]).reset_index(drop=True).T for r in range(run)])

### And to assess the value of the MAE for the test functions

In [4]:
for in1,n1 in enumerate(n):
    run_samples = []

    MAE_dic = {f.__name__:pd.DataFrame(columns=[r for r in range(run)]) for f in functions}
    RMSE_dic = {f.__name__:pd.Series() for f in functions}
    SE_dic = {f.__name__:[] for f in functions}
    for r in range (run):
        run_samples.append(df.iloc[int(r*(len(df)/run)):int((r+1)*(len(df)/run))].reset_index(drop=True))

        sample_Matrices = [run_samples[-1].iloc[:,m*k:(m+1)*k].T.reset_index(drop=True).T for m in range(n1)]

        mixed_Matrices = []
        elementary_effects_list =[]
        f_elementary = []
        for im,m in enumerate(range(n1)):
            elementary_effects_dic = {j:[] for j in range(k)}
            f_elementary.append({f.__name__:pd.DataFrame(columns=[j for j in range(k)]) for f in functions})
            for j in range(k):
                elementary_effects = [sample_Matrices[m]]
                for q in range(n1-1):
                    mixed_Matrices.append(sample_Matrices[m].copy())
                    mixed_Matrices[-1][j]=sample_Matrices[np.roll(np.arange(n1),(n1-1)*m)[q+1]][j]
                    
                    elementary_effects.append(mixed_Matrices[-1])
                elementary_effects_dic[j]=elementary_effects
                
            elementary_effects_list.append(elementary_effects_dic)
            
        for f in functions:
            for ie,e in enumerate(elementary_effects_list):
                for j in range(k):
                    el = []
                    for q in range(len(e[j])):
                        for qi in range(q+1,len(e[j])):
                            el.append(0.5*(f(e[j][q])-f(e[j][qi]))**2)
                    f_elementary[ie][f.__name__][j] = pd.concat(el)
                C_T = pd.concat([fe[f.__name__] for fe in f_elementary]).sort_index().expanding(1).mean()
            Var = f(pd.concat(sample_Matrices).sort_index()).expanding(1).var(ddof=0).expanding(1).mean()
            T = (C_T[~C_T.index.duplicated(keep='last')].T/Var[~Var.index.duplicated(keep='last')]).T
            AE_r = np.abs(T - AE_df.loc[f.__name__])
            MAE_r = AE_r.mean(axis=1)
            MAE_r.index=(MAE_r.index+1)*len(sample_Matrices)*(1+k*(len(sample_Matrices)-1))
            MAE_dic[f.__name__][r]=MAE_r
            
    for f in functions:
        RMSE_dic[f.__name__]= np.sqrt(pd.concat(SE_dic[f.__name__],axis=1).groupby(pd.concat(SE_dic[f.__name__],axis=1).columns,
                                                       1).mean()).mean(axis=1)
            
    writer = pd.ExcelWriter(str(n1)+'Multiple_matrices.xlsx', engine='xlsxwriter')

    for mk in MAE_dic.keys():
        MAE_dic[mk].mean(axis=1).to_excel(writer, sheet_name=mk)
    
    writer.save()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


ValueError: No objects to concatenate

In [ ]:
ls = ['-',':','-.','--',(0, (3, 5, 1, 5, 1, 5))]
mk = ['s','o','v','+','*']

for f in functions:
    Saltenis = pd.read_excel('Saltenis_asym.xlsx',sheet_name=f.__name__,index_col=0).loc[[(k+1)*(-1+2**s) for s in 
                                                                                        range(2,p+1)]].loc[:(k+1)*2**(p+1)]
    Saltenis_sym = pd.read_excel('2Multiple_matrices.xlsx',sheet_name=f.__name__,index_col=0).loc[[2*(k+1)*(-1+2**s) for s in range(2,
                                                                                        p+1)]].loc[:(k+1)*2**(p+1)]
    Matrices3 = pd.read_excel('3Multiple_matrices.xlsx',sheet_name=f.__name__,index_col=0).loc[[(3*(1+k*(3-1)))*(-1+2**s) for s in range(2,
                                                                                        p+1)]].loc[:(k+1)*2**(p+1)]
    Matrices4 = pd.read_excel('4Multiple_matrices.xlsx',sheet_name=f.__name__,index_col=0).loc[[(4*(1+k*(4-1)))*(-1+2**s) for s in range(2,
                                                                                        p+1)]].loc[:(k+1)*2**(p+1)]
    Matrices6 = pd.read_excel('6Multiple_matrices.xlsx',sheet_name=f.__name__,index_col=0).loc[[(6*(1+k*(6-1)))*(-1+2**s) for s in range(2,
                                                                                        p+1)]].loc[:(k+1)*2**(p+1)]
    
    dataSets = [Saltenis,Saltenis_sym,Matrices3,Matrices4,Matrices6]

    dSnames = ['Saltenis','Saltenis_sym','Saltenis_3_matrices','Saltenis_4_matrices','Saltenis_6_matrices']
    
    dS_dic = dict(zip(dSnames,dataSets))
    
    co = -1
    for dk, dv in dS_dic.items():
        co +=1
        x_vals = dv.index
        y1 = dv.values
        plt.loglog(x_vals, y1, c='k', marker = mk[co],linestyle=ls[co],label = dk)
    plt.xlabel('Cost')
    plt.ylabel('MAE')
    plt.title(f.__name__)
    plt.legend()
    plt.show()